# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,26.17,82,0,3.09,US,1699037407
1,1,ribeira grande,38.5167,-28.7000,21.22,94,75,9.26,PT,1699037407
2,2,kloulklubed,7.0419,134.2556,25.28,75,79,1.93,PW,1699037408
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,4.14,64,75,6.26,RU,1699037408
4,4,kodiak,57.7900,-152.4072,7.73,87,100,5.14,US,1699037408


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size='Humidity',
    scale = 1.0,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 400,
    alpha=0.5
    
    
)

# Display the map
city_map_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values

# Max Temp between 20-25. Humidity between 70 and 80
cool_city_df=city_data_df.loc[(city_data_df["Max Temp"]<=25) &
                               (city_data_df["Max Temp"]>=20) &
                               (city_data_df["Humidity"]>=70) &
                                (city_data_df["Humidity"]<=80)]

# Drop any rows with null values
cool_city_df=cool_city_df.dropna()

# Display sample data
cool_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,callao,-12.0667,-77.1500,21.16,73,75,4.12,PE,1699037353
103,103,pisco,-13.7000,-76.2167,22.23,73,100,4.12,PE,1699037337
121,121,chenzhou,25.8000,113.0333,22.81,79,100,1.57,CN,1699037430
128,128,dondo,-19.6094,34.7431,23.73,80,49,3.28,MZ,1699037431
152,152,port douglas,-16.4833,145.4667,23.91,77,100,2.58,AU,1699037436
161,161,puerto santa,-8.9906,-78.6456,20.99,75,16,4.87,PE,1699037437
205,205,port mathurin,-19.6833,63.4167,24.44,78,91,4.37,MU,1699037444
212,212,essaouira,31.5125,-9.7700,20.72,73,0,7.72,MA,1699037444
262,262,huarmey,-10.0681,-78.1522,22.06,70,15,4.79,PE,1699037451
275,275,marsh harbour,26.5412,-77.0636,24.68,78,100,11.30,BS,1699037453


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=cool_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

C:\Users\tamhl\AppData\Local\Temp\ipykernel_3300\1827023265.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''


,City,Country,Lat,Lng,Humidity,Hotel Name
14,callao,PE,-12.0667,-77.1500,73,
103,pisco,PE,-13.7000,-76.2167,73,
121,chenzhou,CN,25.8000,113.0333,79,
128,dondo,MZ,-19.6094,34.7431,80,
152,port douglas,AU,-16.4833,145.4667,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "format":"json",
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lon=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
callao - nearest hotel: Casa Ronald
pisco - nearest hotel: La Portada
chenzhou - nearest hotel: 明桂园酒店
dondo - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
puerto santa - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
essaouira - nearest hotel: Essaouira Wind Palace
huarmey - nearest hotel: Hostal Santa Rosa
marsh harbour - nearest hotel: Lofty Fig Villas
nova sintra - nearest hotel: Residência Ka Dencho
bogra - nearest hotel: Ruposhi Bangla Hotel
alofi - nearest hotel: Matavai Resort
paita - nearest hotel: Victor's
burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
tamiahua - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
urangan - nearest hotel: Ramada
katsuura - nearest hotel: 潮騒の宿
petite riviere - nearest hotel: Palais Hans
ha'il - nearest hotel: فندق كلاسيك


,City,Country,Lat,Lng,Humidity,Hotel Name
14,callao,PE,-12.0667,-77.1500,73,Casa Ronald
103,pisco,PE,-13.7000,-76.2167,73,La Portada
121,chenzhou,CN,25.8000,113.0333,79,明桂园酒店
128,dondo,MZ,-19.6094,34.7431,80,No hotel found
152,port douglas,AU,-16.4833,145.4667,77,Mantra Aqueous Port Douglas
161,puerto santa,PE,-8.9906,-78.6456,75,No hotel found
205,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
212,essaouira,MA,31.5125,-9.7700,73,Essaouira Wind Palace
262,huarmey,PE,-10.0681,-78.1522,70,Hostal Santa Rosa
275,marsh harbour,BS,26.5412,-77.0636,78,Lofty Fig Villas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Configure map

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"]    

)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)